# ***Capstone Part 1: Segmenting and Clustering Neighborhoods in Toronto***

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

!pip3 install BeautifulSoup4
!pip install lxml
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: d

## Downloading Data

In [ ]:
data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") #read page into dataframe

In [ ]:
print(len(data)) #number of tables in the page

In [ ]:
df = data[0] #used list to locate the table

In [ ]:
df.head()

## Cleaning Data

In [ ]:
df = df[df.Borough != 'Not assigned'] #removing cells with no boroughs

In [ ]:
df = df.reset_index(drop=True) #resetting index after dropping cells

In [ ]:
print(df.shape[0]) #number of rows after removing cells

# ***Capstone Part 2: Segmenting and Clustering Neighborhoods in Toronto***

## Adding Longitudes and Lattitudes

In [ ]:
df2 = pd.read_csv('https://cocl.us/Geospatial_data') #due to errors with the geocoder package used csv to read lat & long
df2.head()

In [ ]:
df3 = df.join(df2.set_index('Postal Code'), on='Postal Code') #joined the two data frames on postal code to get latitude and longitude for each postal code
df3.head()

# ***Capstone Part 3: Segmenting and Clustering Neighborhoods in Toronto***

## Explore and cluster

#### Create a map of New York with neighborhoods superimposed on top.

In [ ]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Exploring neighborhoods in Downtown Toronto.

In [ ]:
DT_data = df3[df3['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
DT_data.head()

In [ ]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of DT using latitude and longitude values
map_DT = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(DT_data['Latitude'], DT_data['Longitude'], DT_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_DT)  
    
map_DT

###  Segment Neighborhoods

In [ ]:
CLIENT_ID = 'HALZBJO31KAFZT3EWQ1Y3BPTTLJDNSLWUGAJOT1YOGWZKOQG'
CLIENT_SECRET = '0212FY1LTCNDLLA4TU0IYYCTCMA5SZKRYXAXP4JZYKAWNPLU'
VERSION = '20180605' # Foursquare API version

In [ ]:
neighborhood_latitude = DT_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = DT_data.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = DT_data.loc[0, 'Neighborhood'] # neighborhood name

In [ ]:
LIMIT = 100 # limit of number of venues returned by Foursquare API


radius = 500 # define radius


# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

In [ ]:
results = requests.get(url).json()
results

In [ ]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

nearby_venues.head()

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
DT_venues = getNearbyVenues(names=DT_data['Neighborhood'],
                                   latitudes=DT_data['Latitude'],
                                   longitudes=DT_data['Longitude']
                                  )

In [ ]:
print(DT_venues.shape) #how many venues were returned for each neighborhood
DT_venues.head()

In [ ]:
DT_venues.groupby('Neighborhood').count() #how many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} uniques categories.'.format(len(DT_venues['Venue Category'].unique())))

### Analyze Each Neighborhood

In [ ]:
# one hot encoding
DT_onehot = pd.get_dummies(DT_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
DT_onehot['Neighborhood'] = DT_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [DT_onehot.columns[-1]] + list(DT_onehot.columns[:-1])
DT_onehot = DT_onehot[fixed_columns]

DT_onehot.head()

In [ ]:
DT_onehot.shape #the new dataframe size.

In [ ]:
DT_grouped = DT_onehot.groupby('Neighborhood').mean().reset_index() #the frequency of occurrence of each category
DT_grouped

In [ ]:
DT_grouped.shape #updated size

In [ ]:
num_top_venues = 5 #the top 5 most common venues

for hood in DT_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = DT_grouped[DT_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = DT_grouped['Neighborhood']

for ind in np.arange(DT_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DT_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

### Cluster Neighborhoods

In [ ]:
# set number of clusters
kclusters = 5

DT_grouped_clustering = DT_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(DT_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

DT_merged = DT_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
DT_merged = DT_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

DT_merged.head()

In [ ]:
# create map finalize clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(DT_merged['Latitude'], DT_merged['Longitude'], DT_merged['Neighborhood'], DT_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

Cluster 1

In [ ]:
DT_merged.loc[DT_merged['Cluster Labels'] == 0, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

Cluster 2

In [ ]:
DT_merged.loc[DT_merged['Cluster Labels'] == 1, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

Cluster 3

In [ ]:
DT_merged.loc[DT_merged['Cluster Labels'] == 2, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

Cluster 4

In [ ]:
DT_merged.loc[DT_merged['Cluster Labels'] == 3, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

Cluster 5

In [ ]:
DT_merged.loc[DT_merged['Cluster Labels'] == 4, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]